In [1]:
!pip install langchain pypdf

In [7]:
# Helper function for printing docs
import os
OPENAI_API_KEY="sk-..."
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY
def pretty_print_docs(docs):
    print(f"\n{'-' * 100}\n".join([f"Document {i+1}:\n\n" + d.page_content for i, d in enumerate(docs)]))


In [8]:
# The regular way to get the relevant documents
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.document_loaders import PyPDFLoader
from langchain.vectorstores import FAISS


loader = PyPDFLoader("data/Best-Of-MASS-2021-2022.pdf")
pages = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(pages)
retriever = FAISS.from_documents(texts, OpenAIEmbeddings()).as_retriever()

docs = retriever.get_relevant_documents("What are the latest findings of cold exposure and its effects on the fat loss?")
pretty_print_docs(docs)

Document 1:

You’re currently reading MASS Research 
Review, so I assume we can generally agree 
that science is pretty cool. A positive aspect 
of this perspective is that we can embrace 
rigorous scientific principles and use them 
to optimize our habits and practices related 
to training, nutrition, and other health behav-
iors. There is, however, a potential drawback 
of this perspective. If we get a little too en-
thusiastic about how cool science is, we can 
sometimes get too enamored with fascinating 
aspects of human physiology. If we aren’t 
careful, we can get sucked into overhyped 
concepts that encourage us to put the cart be-
fore the horse (i.e., apply an intriguing inter -
vention before we have evidence to suggest 
that it’s actually applicable). Not everything 
that is fascinating is actionable, and not ev-
erything that is actionable is fascinating. 
That brings us to the concept of cold expo-
sure. I’m not talking about cold water im-
mersion to reduce inflammation f

In [9]:
# Gettng relevant documents using compression
from langchain.llms import OpenAI
from langchain import LLMChain
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor
from langchain.prompts import PromptTemplate


template = """
I want you to act as a research assistant. You will answer questions about the context.

Context: {context}
Question: {question}?
"""

prompt = PromptTemplate(
    input_variables=["question", "context"],
    template=template,
)

llm = OpenAI(temperature=0)
compressor = LLMChainExtractor.from_llm(llm)
compression_retriever = ContextualCompressionRetriever(base_compressor=compressor, base_retriever=retriever)

question = "What are the latest findings of cold exposure and its effects on fat loss?"

compressed_docs = compression_retriever.get_relevant_documents(question)
pretty_print_docs(compressed_docs)

llm_chain = LLMChain(prompt=prompt, llm=llm, verbose=True)
llm_chain.predict(question=question, context=compressed_docs)







Document 1:

"One of the purported outcomes most frequently discussed is increased energy expenditure and fat oxidation, which has caused some fairly notable figures to suggest that frequent cold exposure is an effective way to facilitate fat loss."
----------------------------------------------------------------------------------------------------
Document 2:

The presenter suggests that cold water immersion is a viable fat loss tool, but that you should be mindful not to adapt to it because of anecdotes involving cold-water swimmers with high body-fat levels. The presently reviewed study directly contradicts these recommendations; winter swimmers tended to be leaner than well-matched controls, and had more robust thermogenic responses to cold exposure. Cold-induced increases in energy expenditure of nearly 50% (relative to thermoneutral energy expenditure), but this is far from the norm. Other studies often report values around the 15% range (2, 3), with a high degree of variability 

'\nAnswer: The latest findings of cold exposure and its effects on fat loss suggest that cold exposure can increase energy expenditure and fat oxidation, but the degree of increase varies from person to person. Studies have also found that individuals who spend a lot of time in cold environments have upregulated brown adipose tissue activity, and that some of their white adipose tissue may start to behave more like brown adipose tissue. This semi-converted fat is often referred to as “beige” adipose tissue, and the potential to intentionally induce this conversion has spurred interest in studying chronic cold exposure.'